<a href="https://colab.research.google.com/github/JiseokSeo/-I-studied-/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC1_%EB%B9%88%EB%8F%84%EB%A1%9C_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 인사이트

점수가 많이 변하지는 않았다. (0.86 -> 0.88)  
접근 방법 자체는 나쁘지 않은 듯.  
아마 지저분한 데이터를 만나게 된다면 성능이 오를 것 같다.  
직접 메모장을 열어보니 new_title이 공백인 경우가 있었다.  
이를 보정하기 위해 적절한 커트라인 설정이 필요해보인다.
보정이 된다면 점수가 더 오를 것 같다. 

## 후속연구  
접근법 자체는 틀린 것 같지 않아보인다.  
과감하게 title을 건들여도 성능에 큰 악영향이 없다.  
최빈값 순으로 짤랐지만, 뽀로로나 td-idf같은 벡터화 기법을 사용하면 성능이 더 오를 것으로 보인다.



### 핵심 아이디어  :
키워드 N개를 제외하고 전부 삭제한다

1. 키워드를 어떻게 찾을 것인가?  
  ㄱ. td-idf  
  ㄴ. 뽀로로 라이브러리   
  ㄷ. 벡터라이즈하고 벡터의 평균  
  ㄹ. 최빈값 순으로 자르기  

2. 키워드를 찾되 무엇과 비교하여 키워드를 찾아야 하는가?  
  ㄱ. 타이틀 - 카테고리  
  ㄴ. 타이틀 - 카테고리 전체 벡터의 평균  
  ㄷ. 타이틀 내에서 비교  
  ㄹ. 타이틀 전체 벡터의 평균  

3. 키워드를 몇 개 남겨야 하는가?  
  ㄱ. 실험적으로 하나씩 늘려본다  
  ㄴ. 그러나 카테고리 별로 다르게 접근한다  

4. 어떻게 평가할 것인가?  
  ㄱ. 기본적으로 정확도를 본다  
  ㄴ. 오차행렬을 그려 특히 못 분류하는 내용을 중점으로 개선작업  

In [503]:
# !pip install --upgrade pip
# !pip install konlpy
# !pip install pororo
# !pip install fasttext
# !pip install hgtk
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [504]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [264]:
# 우선 코코아 데이터셋으로만 연구를 진행해보겠습니다
# 원시데이터들을 하나씩 불러온 흔적들입니다

df3 = pd.read_csv('cocoa.csv', encoding='utf-8', index_col=0)
df3 = pd.read_csv('coffee.csv', encoding='utf-8', index_col=0)
df3 = pd.read_csv('mealkit.csv', encoding='utf-8', index_col=0)
# df4 = pd.read_csv('naverAPI.csv', encoding='utf-8', index_col=0)
df5 = pd.read_csv('rice.csv', encoding='utf-8', index_col=0)
df6 = pd.read_csv('salad.csv', encoding='utf-8', index_col=0)
df11 = pd.read_csv('simple.csv', encoding='utf-8', index_col=0)
df11 = pd.read_csv('snack.csv', encoding='utf-8', index_col=0)
df11 = pd.read_csv('tea1.csv', encoding='utf-8', index_col=0)
df30 = pd.read_csv('tea2.csv', encoding='utf-8', index_col=0)
df31 = pd.read_csv('water.csv', encoding='utf-8', index_col=0)

In [363]:
#1, 최빈값으로 키워드 검색해보기
import nltk
from konlpy.tag import Okt
from tqdm import tqdm
t = Okt()

# 명사만 추출
titles = df11['title'].tolist()
titles = [t.nouns(ch) for ch in tqdm(titles)]

100%|██████████| 20145/20145 [00:35<00:00, 570.55it/s]


In [364]:
titles[0:9]

[['천년', '동안', '해양심층수', '에코', '개입', '무', '라벨', '천년', '건강', '생수', '천년', '동안'],
 ['천년', '동안', '해양심층수', '에코', '개입', '무', '라벨', '천년', '건강', '생수', '천년', '동안'],
 ['평점', '경도', '미네랄', '워터', '음료', '닥터'],
 ['스파', '무', '라벨'],
 ['스파', '무', '라벨'],
 ['에프', '앤비', '동원', '샘물'],
 ['광동', '직영', '삼다수', '개입', '유', '무', '라벨', '랜덤', '발송'],
 ['농심', '산수', '무', '라벨'],
 ['에프', '앤비', '동원', '샘물', '무', '라벨']]

In [365]:
# 명사들 빈도 파악
count_dict = {}
for i in tqdm(titles):
  for j in i:
    if j not in count_dict:
      count_dict[j] = 1
    else:
      count_dict[j] += 1

100%|██████████| 20145/20145 [00:00<00:00, 204007.64it/s]


In [366]:
# 빈도를 파악하여 일정 빈도수 이하의 단어들을 모두 거른다 
items = count_dict.items()
sorted_items = sorted(items, key=lambda x: x[1], reverse=True)

cut_line = int(len(sorted_items) / 20) # 빈도 상위 5퍼센트의 단어만 선택
outliner_vocab = [k for k, v in sorted_items[:cut_line]]

In [367]:
outliner_vocab[:10] # 거름사전

['개', '제로', '캔', '팩', '두유', '병', '우유', '음료', '콩', '박스']

In [368]:
# '거름 사전'에 없는 단어들 모두 삭제
new_titles = []

for title in titles:
  temp_list = []

  for vocab in title:
    if vocab in outliner_vocab:
      temp_list.append(vocab)

  new_titles.append(temp_list)

In [369]:
# 아웃라이어 단어가 걸러진 제목들 비교
new_titles[0:4], titles[0:4]

([['개입', '무', '라벨', '건강', '생수'],
  ['개입', '무', '라벨', '건강', '생수'],
  ['미네랄', '워터', '음료'],
  ['스파', '무', '라벨']],
 [['천년', '동안', '해양심층수', '에코', '개입', '무', '라벨', '천년', '건강', '생수', '천년', '동안'],
  ['천년', '동안', '해양심층수', '에코', '개입', '무', '라벨', '천년', '건강', '생수', '천년', '동안'],
  ['평점', '경도', '미네랄', '워터', '음료', '닥터'],
  ['스파', '무', '라벨']])

In [370]:
# df 최신화
df11['new_title'] = new_titles

In [422]:
df11.head(3)

,title,cat1,cat2,price,new_title
0,[천년동안] 해양심층수 에코 350ml 20개입 무라벨 천년을 돌아 깨끗함을 넘어 ...,생수/음료,생수,"16,000원","[단위단위, 무, 라벨, 건강, 생수]"
1,[천년동안] 해양심층수 에코 1.2L 9개입 무라벨 천년을 돌아 깨끗함을 넘어 건강...,생수/음료,생수,"12,240원","[단위단위, 무, 라벨, 건강, 생수]"
2,평점 4.9 경도210 미네랄워터 음료 닥터M,생수/음료,생수,"99,500원","[미네랄, 워터, 음료]"


In [372]:
def remove_specific_string(x):
    if isinstance(x, list):
        return [re.sub(r'개|팩|입|봉|병|봉지|잔|상자|박스|BOX|box|Box|ea', '단위', val) for val in x]
    return x

df11['new_title'] = df11['new_title'].apply(remove_specific_string)

In [373]:
df11['new_title'][0]

['단위단위', '무', '라벨', '건강', '생수']

In [374]:
df11.head(3)

,title,cat1,cat2,price,new_title
0,[천년동안] 해양심층수 에코 350ml 20개입 무라벨 천년을 돌아 깨끗함을 넘어 ...,생수/음료,생수,"16,000원","[단위단위, 무, 라벨, 건강, 생수]"
1,[천년동안] 해양심층수 에코 1.2L 9개입 무라벨 천년을 돌아 깨끗함을 넘어 건강...,생수/음료,생수,"12,240원","[단위단위, 무, 라벨, 건강, 생수]"
2,평점 4.9 경도210 미네랄워터 음료 닥터M,생수/음료,생수,"99,500원","[미네랄, 워터, 음료]"


In [421]:
## 전체를 자동화하는법을 몰라서 일단 노가다를 해봤다.
df = pd.concat([df1, df2, df3, df5, df6, df7, df8, df9, df10, df11])
df.reset_index(inplace=True)

In [423]:
df.head(3)

,index,title,cat1,cat2,price,new_title
0,0,스위스미스 다크 초코 코코아 280g 12개,커피/차류,코코아,"50,940원","[스위스, 미스, 초코, 코코아, 단위]"
1,1,기라델리 프리미엄 핫 코코아믹스 초콜렛 907g,커피/차류,코코아,"16,980원","[핫, 코코아, 믹스, 초콜렛]"
2,2,네슬레 네스퀵 초콜렛파우더 1.2kg 2개세트,커피/차류,코코아,"14,500원","[네슬레, 네스퀵, 초콜렛, 파우더, 단위, 세트]"


In [458]:
df_train, df_test = train_test_split(df,
                                     test_size=0.1,
                                     stratify=df['cat2'],
                                     random_state=42)
df_train, df_valid = train_test_split(df_train,
                                      test_size=0.3,
                                      stratify=df_train['cat2'],
                                      random_state=42)

In [459]:
train_token = df_train['new_title']
val_token = df_valid['new_title']
test_token = df_test['new_title']

In [465]:
with open('train_data.txt', 'w') as out:
  for i in tqdm(range(len(train_token)), unit=' line'):
    out.write("__label__" + df_train['cat2'].iloc[i] + "\t" + ' '.join(train_token.iloc[i]) + '\n')

with open('val_data.txt', 'w') as out:
  for i in tqdm(range(len(val_token)), unit=' line'):
    out.write("__label__" + df_valid['cat2'].iloc[i] + "\t" + ' '.join(val_token.iloc[i]) + '\n')

with open('test_data.txt', 'w') as out:
  for i in tqdm(range(len(test_token)), unit=' line'):
    out.write("__label__" + df_test['cat2'].iloc[i] + "\t" + ' '.join(test_token.iloc[i]) + '\n')

100%|██████████| 12416/12416 [00:00<00:00, 55249.22 line/s]


In [477]:
import fasttext

model1 = fasttext.train_supervised(input='train_data.txt',
                                   autotuneValidationFile='val_data.txt')

In [478]:
model1.test('test_data.txt')
# (12416, 0.8807989690721649, 0.8807989690721649)

(12416, 0.8807989690721649, 0.8807989690721649)

In [ ]:
# 오차행렬
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

df_test['predict'] = df_test['predict'] = df_test['title'].apply(lambda x: str(model1.predict(x)[0][0])).str.replace('__label__', '')

plt.rc('font', family='NanumBarunGothic')
f, ax = plt.subplots(figsize=(24, 24))
cm = confusion_matrix(df_test['predict'], df_test['cat2'], normalize='true')
disp = ConfusionMatrixDisplay(cm, display_labels=df_test['cat2'].unique())
disp.plot(cmap='Blues', values_format='.2f', ax=ax, colorbar=False)

In [509]:
# 베이스라인과 비교하기
train_token_base = df_train['title']
val_token_base = df_valid['title']
test_token_base = df_test['title']

with open('train_data_base.txt', 'w') as out:
  for i in tqdm(range(len(train_token)), unit=' line'):
    out.write("__label__" + df_train['cat2'].iloc[i] + "\t" + ''.join(train_token_base.iloc[i]) + '\n')

with open('val_data_base.txt', 'w') as out:
  for i in tqdm(range(len(val_token)), unit=' line'):
    out.write("__label__" + df_valid['cat2'].iloc[i] + "\t" + ''.join(val_token_base.iloc[i]) + '\n')

with open('test_data_base.txt', 'w') as out:
  for i in tqdm(range(len(test_token)), unit=' line'):
    out.write("__label__" + df_test['cat2'].iloc[i] + "\t" + ''.join(test_token_base.iloc[i]) + '\n')

100%|██████████| 12416/12416 [00:00<00:00, 51717.10 line/s]


In [511]:
# 베이스라인
model_base = fasttext.train_supervised(input='train_data_base.txt',
                                       autotuneValidationFile='val_data_base.txt')

model_base.test('test_data_base.txt')

(12416, 0.8604220360824743, 0.8604220360824743)

In [ ]:
# 오차행렬 
df_test['predict_base'] = df_test['title'].apply(lambda x: str(model_base.predict(x)[0][0])).str.replace('__label__', '')

plt.rc('font', family='NanumBarunGothic')
f, ax = plt.subplots(figsize=(24, 24))
cm = confusion_matrix(df_test['predict'], df_test['cat2'], normalize='true')
disp = ConfusionMatrixDisplay(cm, display_labels=df_test['cat2'].unique())
disp.plot(cmap='Blues', values_format='.2f', ax=ax, colorbar=False)